In [1]:
import importlib
import segment
importlib.reload(segment)
from segment import load_report_with_images, get_docs_with_ocr
from doctr.models import ocr_predictor
import os
import pickle

CACHE_PATH = "../data/cache/ocr_docs3.pkl"

if os.path.exists(CACHE_PATH):
    print("Loading docs from cache...")
    with open(CACHE_PATH, "rb") as f:
        docs = pickle.load(f)
    print("Loaded from cache:", len(docs))
else:
    ocr_model = ocr_predictor(det_arch='db_resnet50', reco_arch='crnn_vgg16_bn', pretrained=True).cuda()
    items = load_report_with_images(limit=32)
    print("Loaded:", len(items))
    docs = get_docs_with_ocr(items, ocr_model)
    os.makedirs(os.path.dirname(CACHE_PATH), exist_ok=True)
    with open(CACHE_PATH, "wb") as f:
        pickle.dump(docs, f)
    print("Saved docs to cache:", CACHE_PATH)

docs = [doc for doc in docs if len(doc['segments']) < 120]
print("Processed Docs:", len(docs))

Loading docs from cache...
Loaded from cache: 995
Processed Docs: 995


In [10]:
import os
import json

def convert_segments_to_yolo(docs, out_dir="../data/yolo_segments"):
    os.makedirs(f"{out_dir}/images", exist_ok=True)
    os.makedirs(f"{out_dir}/labels", exist_ok=True)

    for idx, doc in enumerate(docs):
        img_path = doc["image_path"]
        segments = doc["segments"]
        H, W = doc["dimensions"]

        # Copy image
        img_out = f"{out_dir}/images/{idx}.jpg"
        import shutil
        shutil.copy2(img_path, img_out)

        # Write YOLO label file (all segments = class 0)
        label_out = f"{out_dir}/labels/{idx}.txt"
        with open(label_out, "w") as f:
            for seg in segments:
                box = seg["box"]
                x_min = box["x_min"]
                y_min = box["y_min"]
                x_max = box["x_max"]
                y_max = box["y_max"]

                # Normalize YOLO format
                x_center = (x_min + x_max) / 2 / W
                y_center = (y_min + y_max) / 2 / H
                w = (x_max - x_min) / W
                h = (y_max - y_min) / H

                # YOLO only needs class_id=0 for all
                f.write(f"0 {x_center} {y_center} {w} {h}\n")

    print("YOLO segment-only dataset created!", out_dir)

convert_segments_to_yolo(docs)

YOLO segment-only dataset created! ../data/yolo_segments


In [ ]:
# train_yolo.py
from doclayout_yolo import YOLOv10

def train_yolo():
    model = YOLOv10("../models/yolo.pt")   # lightweight and fast

    model.train(
        data="../data/yolo_segments.yaml",
        epochs=50,
        imgsz=1024,
        batch=8,
        device="cuda",
        lr0=0.001,
        optimizer="AdamW",
        patience=20,
    )

    model.save("../models/yolo_segments_trained.pt")
    print("Training complete! Saved model as yolo_segments_trained.pt")

train_yolo()

Ultralytics YOLOv0.0.4 🚀 Python-3.13.5 torch-2.9.1+cu128 CUDA:0 (NVIDIA A100-SXM4-80GB, 81038MiB)
engine/trainer: task=detect, mode=train, model=../models/yolo.pt, data=../data/yolo_segments.yaml, epochs=50, time=None, patience=20, batch=8, imgsz=1024, save=True, save_period=10, val_period=1, cache=False, device=cuda, workers=8, project=None, name=train, exist_ok=True, pretrained=True, optimizer=AdamW, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_lab

AMP: running Automatic Mixed Precision (AMP) checks with YOLOv8n...
AMP: checks skipped ⚠️, offline and unable to download YOLOv8n. Setting 'amp=True'. If you experience zero-mAP or NaN losses you can disable AMP with amp=False.


/home/compiling-ganesh/24m0797/workspace/doctr-dit/.venv/lib/python3.13/site-packages/doclayout_yolo/engine/trainer.py:277: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(enabled=self.amp)
train: Scanning /home/compiling-ganesh/24m0797/workspace/doctr-dit/data/yolo_segments/labels... 995 images, 0 backgrounds, 1 corrupt: 100%|██████████| 995/995 [00:00<00:00, 1043.17it/s]

train: WARNING ⚠️ /home/compiling-ganesh/24m0797/workspace/doctr-dit/data/yolo_segments/images/145.jpg: ignoring corrupt image/label: non-normalized or out of bounds coordinates [1.1913536 1.0420649 1.1729534 1.125281  1.1158383 1.1422107 1.1216625
 1.1487547 1.1511207 1.132253  1.1215011 1.1261437 1.1661302 1.114101
 1.1600661 1.1063203 1.1306188]


train: New cache created: /home/compiling-ganesh/24m0797/workspace/doctr-dit/data/yolo_segments/labels.cache
albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, method='weighted_average', num_output_channels=3), CLAHE(p=0.01, clip_limit=(1.0, 4.0), tile_grid_size=(8, 8))


...

      12/50      17.8G      0.984     0.5998          0      1.192     0.6723          0         65       1024: 100%|██████████| 125/125 [00:40<00:00,  3.08it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 63/63 [00:14<00:00,  4.36it/s]


994
                   all        994      19299      0.882      0.862      0.906      0.644

      Epoch    GPU_mem     box_om     cls_om     dfl_om     box_oo     cls_oo     dfl_oo  Instances       Size


      13/50      17.5G     0.9618     0.5842          0      1.174     0.6529          0        146       1024: 100%|██████████| 125/125 [00:39<00:00,  3.13it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 63/63 [00:14<00:00,  4.44it/s]


994
                   all        994      19299      0.892      0.864      0.913      0.649

      Epoch    GPU_mem     box_om     cls_om     dfl_om     box_oo     cls_oo     dfl_oo  Instances       Size


      14/50        18G     0.9224     0.5625          0      1.129     0.6167          0        301       1024:  96%|█████████▌| 120/125 [00:38<00:01,  3.06it/s]